In [2]:
import os

In [3]:
%pwd

'd:\\End-End DS Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\End-End DS Project'

In [26]:
from dataclasses import dataclass
from pathlib import Path
import yaml

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    load_data_file: Path
    unzip_dir: Path


In [27]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml, create_directories

## Create the configuration manager class to manage the configuration files
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, 
                       params_filepath = PARAMS_FILE_PATH, 
                       schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_URL,
            load_data_file= config.load_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [28]:
## update component to use the new config

from urllib.request import urlretrieve
import urllib.request as request
from src.DataScience import logger
import zipfile

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.load_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.load_data_file
                )
            logger.info(f"File {filename} downloaded with headers: {headers}")
        else:
            logger.info(f"File {self.config.load_data_file} already exists. Skipping download.")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.load_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) 

    

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2025-04-15 12:16:51,277: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2025-04-15 12:16:51,278: INFO: common: yaml file: params.yaml loaded sucessfully]
[2025-04-15 12:16:51,280: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2025-04-15 12:16:51,280: INFO: common: Creating a Directory at: artifacts]
[2025-04-15 12:16:51,280: INFO: common: Creating a Directory at: artifacts/data_ingestion]
[2025-04-15 12:16:52,118: INFO: 1022936704: File artifacts/data_ingestion/data.zip downloaded with headers: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1946:20CBD3:3C0A55:A63E95:67FE00DB
Accept-Ranges: bytes
Date: Tue, 1